In [2]:
import pandas as pd
import numpy as np

/home/yuto/m5-forecasting-accuracy/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
file_name = './output/20200408_070832/test.csv'
predict = pd.read_csv(file_name, usecols=['demand'])
predict.head()

,demand
0,0.845097
1,0.419935
2,0.485555
3,1.904490
4,0.921281


In [4]:
# マイナスを除く
predict['demand'] = predict['demand'].map(lambda x: np.maximum(0, x))
predict.head()

,demand
0,0.845097
1,0.419935
2,0.485555
3,1.904490
4,0.921281


In [5]:
predict[predict['demand'] < 0].count()

demand    0
dtype: int64

In [6]:
# data = pd.read_pickle('./resources/feature/feature.pkl').reset_index()

# test = data[(data["date"] > "2016-04-24")].reset_index()
# test = test[['id', 'date', 'demand']]

test = pd.read_csv('./test.csv')
test.head()

,id,date,demand
0,HOBBIES_1_001_CA_1_validation,2016-04-25,0
1,HOBBIES_1_002_CA_1_validation,2016-04-25,0
2,HOBBIES_1_003_CA_1_validation,2016-04-25,0
3,HOBBIES_1_004_CA_1_validation,2016-04-25,0
4,HOBBIES_1_005_CA_1_validation,2016-04-25,0


In [7]:
test['demand'] = predict['demand']
test.head()

,id,date,demand
0,HOBBIES_1_001_CA_1_validation,2016-04-25,0.845097
1,HOBBIES_1_002_CA_1_validation,2016-04-25,0.419935
2,HOBBIES_1_003_CA_1_validation,2016-04-25,0.485555
3,HOBBIES_1_004_CA_1_validation,2016-04-25,1.904490
4,HOBBIES_1_005_CA_1_validation,2016-04-25,0.921281


In [8]:
# dark magic
# test.loc[test["id"].str.endswith("validation"), "demand"] = test['demand'] / test.loc[test["id"].str.endswith("validation"), "demand"].mean() * 1.447147
# test.head()

In [9]:
def make_submission(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)

In [10]:
make_submission(test, pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv'))

In [11]:
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m "tuned scaled regression"

100%|██████████████████████████████████████| 20.5M/20.5M [00:03<00:00, 5.41MB/s]
Successfully submitted to M5 Forecasting - Accuracy